In [8]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder 
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay,multilabel_confusion_matrix
import matplotlib.pyplot as plt
import pickle
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

In [58]:
final_arr = np.load(f'C:/Users/leose/homework/malawi_news_classification/data/'+'split_text_embeddings' '.npy')

labels=pd.read_csv(f'C:/Users/leose/homework/malawi_news_classification/data/'+'split_labels'+'.csv')
labels=labels['Label'].values

idx=pd.read_csv(f'C:/Users/leose/homework/malawi_news_classification/data/'+'split_index'+'.csv')
idx=idx['Idx'].values



In [19]:
xgb_model = XGBClassifier() 
lgr_model= LogisticRegression(random_state=0)
rand_forest = RandomForestClassifier(random_state=0)

X=final_arr
y=labels

skf = StratifiedKFold(n_splits=2)
skf.get_n_splits(X,y)


for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    lgr_model=lgr_model.fit(X_train, y_train)
    rand_forest.fit(X_train, y_train)
    xgb_model.fit(X_train, y_train)
    
    
    y_pred = xgb_model.predict(X_test) 
    predictions = [value for value in y_pred]
    accuracy = accuracy_score(y_test, predictions) 
    print('xgb',accuracy)
#     cm=multilabel_confusion_matrix(y_test,predictions)
#     print('xgb',cm)
    
    
    y_pred = lgr_model.predict(X_test) 
    predictions = [value for value in y_pred]
    accuracy = accuracy_score(y_test, predictions) 
    print('lgr',accuracy)
#     cm=multilabel_confusion_matrix(y_test,predictions)
#     print('lgr',cm)

    y_pred = rand_forest.predict(X_test) 
    predictions = [value for value in y_pred]
    accuracy = accuracy_score(y_test, predictions) 
    print('RF',accuracy)
#     cm=multilabel_confusion_matrix(y_test,predictions)
#     print('RF',cm)


C:\Users\leose\anaconda3\envs\data\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\leose\anaconda3\envs\data\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[08:25:38] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
xgb 0.37953640833528446
lgr 0.4167642238351674
RF 0.34441582767501755


C:\Users\leose\anaconda3\envs\data\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\leose\anaconda3\envs\data\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[08:28:10] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
xgb 0.38688524590163936
lgr 0.431615925058548
RF 0.3550351288056206


In [52]:
y_pred=[]
for i in range(0,len(final_arr)):
    y_pred.append(lgr_model.predict([final_arr[i]]))

In [53]:
y_preds=pd.DataFrame(y_pred,columns=['preds'])

In [123]:
final_df=pd.DataFrame(columns=['Preds','Labels','Indexes'])
final_df['Preds']=y_preds['preds'].values.astype(int)

final_df['Labels']=labels
final_df['Indexes']=idx.astype(int)

In [161]:
a=final_df.groupby('Indexes')['Preds'].value_counts()

In [142]:
fin_labels=final_df.groupby('Indexes')['Labels'].count().reset_index()

In [126]:
final_preds=[]
for i in range(0,1435):
    final_preds.append(a[i].idxmax())


In [140]:
final_preds.append(0)

In [149]:
fin_labels['Labels'].values

array([ 9,  7,  2, ...,  4,  8, 11], dtype=int64)

In [152]:
accuracy = accuracy_score(fin_labels['Labels'],final_preds) 
print(accuracy)

0.056406685236768804
